In [1]:
from __future__ import unicode_literals
import youtube_dl
from pytube import YouTube

def youtube2wav_cc(link):

    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
    }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download([link])

    source = YouTube(link)


    en_caption = source.captions.get_by_language_code('zh-TW')

    en_caption_convert_to_srt =(en_caption.generate_srt_captions())

    #print(en_caption_convert_to_srt)
    #save the caption to a file named Output.txt
    text_file = open(source.title + ".txt", "w")
    text_file.write(en_caption_convert_to_srt)
    text_file.close()

In [2]:
link = 'https://www.youtube.com/watch?v=odILWz_035Q'
youtube2wav_cc(link)
title = YouTube(link).title
video_id = YouTube(link).video_id

[youtube] odILWz_035Q: Downloading webpage
[download] Destination: 他變好壯! 跟大謙一起做運動【阿滴日常】-odILWz_035Q.webm
[download] 100% of 7.66MiB in 00:0020MiB/s ETA 00:008
[ffmpeg] Destination: 他變好壯! 跟大謙一起做運動【阿滴日常】-odILWz_035Q.wav
Deleting original file 他變好壯! 跟大謙一起做運動【阿滴日常】-odILWz_035Q.webm (pass -k to keep)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated function get_by_language_code (This object can be treated as a dictionary, i.e. captions['en']).


In [3]:
import os
def mkdir_(path):
    if not os.path.isdir(path):
        os.mkdir(path)
mkdir_(title)
mkdir_(title+'/wav')
mkdir_(title+'/txt')

In [4]:
import wave
with open(title + '.txt', 'r') as fp:
    cc = fp.read() 
time_step = cc.split('\n')[1::4]
text = cc.split('\n')[2::4]
for i in range(0,len(time_step)-5,5):
    #--------------wav---------------------
    strat_list = time_step[i].split(' --> ')[0].split(':')
    sec = strat_list[2].split(',')
    strat = int(strat_list[0]) * 3600 + int(strat_list[1]) * 60 +  int(sec[0]) + int(sec[1]) * 0.001

    end_list = time_step[i+4].split(' --> ')[1].split(':')
    sec = end_list[2].split(',')
    end = int(end_list[0]) * 3600 + int(end_list[1]) * 60 +  int(sec[0]) + int(sec[1]) * 0.001
    
    

    win= wave.open(title +'-'+ video_id + '.wav', 'rb')
    wout= wave.open(title + '/wav/' +str(i) + '.wav', 'wb')

    t0, t1= strat, end # cut audio between one and two seconds
    s0, s1= int(t0*win.getframerate()), int(t1*win.getframerate())
    win.readframes(s0) # discard
    frames= win.readframes(s1-s0)

    wout.setparams(win.getparams())
    wout.writeframes(frames)

    win.close()
    wout.close()
    #------------txt--------------------------
    with open(title + '/txt/'+str(i)+'.txt', 'w') as fp:
        for j in range(5):
            fp.write(text[i+j] + ' ')